In [ ]:
import pymongo
from pymongo import MongoClient
import csv
import os
from petl import fromcsv, look, cut, tocsv 
import numpy as np
from StringIO import StringIO
import sklearn
from sklearn import linear_model
import plotly 
import plotly.plotly as py
import re
import datetime
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from geopy.distance import vincenty
from scipy.stats import norm
from numpy import linspace
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from pylab import plot,show,hist,figure,title
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
from scipy.stats import norm
import scipy.stats
from matplotlib.legend_handler import HandlerLine2D

In [ ]:
conn = pymongo.MongoClient()
#conn = pymongo.MongoClient('localhost', 27017)
db = conn.mydb_complete
fields=['redirect_to_id', 'bathrooms', 'short_last_update_description_ary', 'sqft_price', 'zone2_ids', 'property_type', 'takedown', 'property_id', 'current_photos_count', 'zip', 'current_list_price', 'previous_sqft_price', 'location', 'previous_price', 'status', 'mischaracterized_rental', 'price', 'bedrooms', 'year_built', 'lot_sqft', 'created_at', 'mls_active', 'for_flippers', '_id', 'sqft','orphan_from_mls']

cursor=db.property_updates.find({
        "$and":[
            {"property_type": "home"}, 
              {"mischaracterized_rental":False},
              {"$or": [{"status": "sold"}, {"status": "pending"}]},
              {"sqft": { "$exists": True, "$ne":"", "$ne":0   } },
            {"property_id": { "$exists": True , "$ne":"" }},
              {"price": { "$exists": True , "$ne":"", "$ne":0 }},
              {"year_built": { "$exists": True , "$ne":"", "$ne":0} },
              {"location": { "$exists": True , "$ne":"" } },
            {"created_at": { "$exists": True, "$ne":""  } }, 
             ]},{'redirect_to_id':0,'short_last_update_description_ary':0,'property_type':0,
                 'current_photos_count':0, 'mischaracterized_rental':0,'mls_active':0,
                'orphan_from_mls':0,'_id':0})

In [ ]:
with open('soldPend_home_v2.csv','w') as outfile:
    fields=['bathrooms','sqft_price','takedown', 'property_id','zip','current_list_price','previous_sqft_price','location','previous_price', 'status', 'price','bedrooms','year_built','lot_sqft','created_at','for_flippers','sqft','zone2_ids']
    writer=csv.DictWriter(outfile,fieldnames=fields)
    writer.writeheader()
    for x in cursor:
            writer.writerow(x)
            
fields=['bathrooms','sqft_price','takedown','property_id','zip','current_list_price', 'previous_sqft_price','location','previous_price', 'status', 'price','bedrooms','year_built','lot_sqft','created_at','for_flippers','sqft','zone2_ids']
#for index in np.arange(len(fields)):
    #print str(index) + "," + fields[index]

In [ ]:
import re
import datetime
pattern1 = r'\"\[(.*)'
pattern2 = r'\s+(.*)\]'

rfile = open('soldPend_home.csv', 'r')
line = rfile.readline()  # skip header
wfile = open('soldPend_home_all.csv', 'w')
wfile.write('pre_sqft_price,pre_price,zipcd,propID,numBath,sqft_price,takedown,cur_lis_price,lat,longt,numBed,year_built,lot_sqft,numDays,year,month,forFlip,sqft,price \n')

line = rfile.readline()
ind = 1

dateRef = '2016-6-30 12:00:00'

#while (line and ind<10):
while (line):
    #print 'line = ' + line
    dataVals = line.split(',')
    numBath = dataVals[0]
    sqft_price = dataVals[1]
    takedown = dataVals[2]
    propID = dataVals[3]
    #print 'propID = ' + str(int(propID,16))
    
    zipcd = dataVals[4]
    cur_lis_price = dataVals[5]
    pre_sqft_price = dataVals[6]
    loc1 = dataVals[7]
    loc2 = dataVals[8]
    pre_price = dataVals[9]
    price = dataVals[11]
    numBed = dataVals[12]
    year_built = dataVals[13]
    lot_sqft = dataVals[14]
    dateCreated = dataVals[15]
    forFlip=dataVals[16]
    sqft = dataVals[17]
    #zoneID1 = dataVals[18]
    #print 'debug = ' + dateCreated
    match1 = re.search(pattern1, loc1)
    match2 = re.search(pattern2, loc2)
    if match1:
        lat = match1.group(1)
        longt = match2.group(1)       
        dt2 = datetime.datetime.strptime(dateCreated, "%Y-%m-%d %H:%M:%S")
        dt1 = datetime.datetime.strptime(dateRef, "%Y-%m-%d %H:%M:%S")
        numDays = (dt1-dt2).days # from dateRef
        month=dt2.month
        year=dt2.year
        
        wfile.write(pre_sqft_price+','+pre_price+','+zipcd+','+propID +','+ numBath+','+sqft_price+','+takedown+','+cur_lis_price+','\
                    +lat+','+longt+','+numBed+','+year_built+','\
                    +lot_sqft+','+str(numDays)+','+str(year)+','+str(month)+','+forFlip+','\
                    +sqft+','+price+'\n')
    line = rfile.readline()
    ind = ind+1
rfile.close()   
wfile.close()